<a href="https://colab.research.google.com/github/AissatouPaye/AnomalyDetection.jl/blob/master/VAE_AnoGAN_f_ANoGAN_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VAE , AnoGAN and F-AnoGAN


Use the MNIST Dataset to see how VAE, AnoGAN and F-AnoGAN works as Anomalies detector



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import matplotlib
import os
import cv2

### use MNIST dataset for the GAN models

from keras .datasets import mnist 

from keras.models import Sequential,Model
from keras.layers import Input, Reshape, Dense, Dropout, MaxPooling2D, Conv2D, Flatten
from keras.layers import Conv2DTranspose, LeakyReLU
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from keras import initializers
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import math

from keras.utils. generic_utils import Progbar
from keras.utils.vis_utils import plot_model

from typing import Tuple
from keras.models import load_model
from keras import backend as K
from sklearn.manifold import TSNE
from keras.layers import concatenate

# Google Colab
from google.colab import drive

from sklearn.manifold import TSNE
import copy
from enum import Enum

from keras.callbacks import TensorBoard
import io
from PIL import Image
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from math import floor
from skimage.transform import resize
from scipy.linalg import sqrtm

MNIST DATASET

Download the data and create a traning and test loader


In [10]:
# Train and test data

(X_train,Y_train),(X_test,Y_test)=mnist.load_data()
### Convert image scale to -1 to 1 ####

X_train=(X_train.astype(np.float32)- DataParm.image_size_127_5.value)/DataParm.image_size_127_5.value
X_test=(X_test.astype(np.float32)- DataParm.image_size_127_5.value)/DataParm.image_size_127_5.value

#### Convert to batch size, width, height, channel ####
X_train=X_train[:,:,:,None]
X_test=X_test[:,:,:,None]

#Learn only image with labe 1 ( our positif sample that the generative model have to learn)

X_train=X_train[Y_train==1]
X_test=X_test[Y_test==1]

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
#####Setting the general parameter that will be use to save the models results ###

class DataParm(Enum):
    # VAE Path to save  models, images, and Tensorboard logs
    common_path_VAE = '/content/drive/My Drive/AnoDetect_implementation/VAE'
    ##### Path AnoGAN
    common_path = '/content/drive/My Drive/AnoDetect_implementation/AnoGAN'
    # EfficentAnnoGAN: Path to store the model, images, and Tensorboard logs
    common_path_efficient = '/ content / drive / My Drive /GAN/f-AnoGAN/'
    image_size_127_5 = 127.5
    image_size= X_train.shape[1]
    original_dim = image_size*image_size
    input_shape= (image_size,image_size,1)
    batch_size_64 = 64

In [13]:
X_train[0].shape


### visualize one sample of the data : showing a 1 of the training set ####



(28, 28, 1)

### VAE 
The VAE has a modular design. The encoder, decoder and VAE
are 3 models that share weights. After training the VAE model,
the encoder can be used to generate latent vectors.
The decoder can be used to generate MNIST digits by sampling the
latent vector from a Gaussian distribution with mean = 0 and std = 1.
### Reference
[1] Kingma, Diederik P., and Max Welling.
"Auto-Encoding Variational Bayes."
https://arxiv.org/abs/1312.6114


#####  put the GENERAL ARCHITECTURE  here#####

ENCODER

In [0]:
#### Convolution VAE : layer of encoder and decoder are convolution net #####

####Building the encoder  ######

class EncoderParm(Enum):
      conv2d_filter_64 = 64
      conv2d_kernel_5_5 = (5, 5)
      conv2d_stride_2_2 = (2, 2)
      conv2d_padding_same = 'same'

      leaky_reLU_param_0_2 = 0.2

      conv2d_second_filter_128 = 128
      conv2d_second_kernel_5_5 = (5, 5)
      conv2d_second_stride_2_2 = (2, 2)
      conv2d_second_padding_same = 'same'
      
      dense_1=1
      out_activation='relu'
    
def  Encoder_model():
  """
  Two layers convolution network
  
  Model that output a mean and variance 
  """
  input = Input(shape=EncoderParm.input_shape.value , name='Encoder_X')
  ##### convolution layer 
  conv1= Conv2D(filters=conv2d_filter_64,
                 kernel_size=conv2d_kernel_5_5,
                stride=conv2d_stride_2_2,
                padding='same')(input)
  
  conv1=LeakyRelu(EncoderParm.leaky_reLU_param_0_2.value)(conv1)
  
  conv2=Conv2D(filters = conv2d_second_filter_128,
               kernel_size = conv2d_second_kernel_5_5,
               stride = conv2d_second_stride_2_2,
               padding = 'same')(conv1)
  
  conv2=LeakyRelu(EncoderParm.leaky_reLU_param_0_2.value)(conv2)
  
  # shape info needed to build decoder model

  shape = K.int_shape(conv2)
  
  fc=Flatten()(conv2)
  fc=Dense(EncoderParm.dense_1.value)(fc)
  output=Activation(EncoderParm.out_activation.value)(fc)
  
  #####
  latent_dim=1 ## number of stack layer in the latent space 
  z_mean  =  Dense ( latent_dim ,  activation = 'linear' ) ( hidden ) 
  z_sigma  =  Dense ( latent_dim ,  activation = 'linear' ) ( hidden )
  
  model=Model(inputs=[input],outputs=[output])  
  return Model , shape
 






##### Sampler  ####

class Sampler (self):
  
  """
  Use reparametrizatin trick instead od sampling from Q(Z/X), sample from epsilon - N(0,I)
  
  z = z_mean+ sqrt(z_var)* epsilon
  """